In [8]:
import numpy as np
import logging
import pandas as pd

logging.basicConfig(level= logging.INFO, format="%(asctime)s %(levelname)s %(message)s", datefmt="%d-%m-%Y")
logger = logging.getLogger()


In [10]:
cdpath = 'data/casing_diameter.csv'
cd = pd.read_csv(cdpath)

,inches,metres,Next biggest casing,Recommended bit
0,0.000,0.000000,0.1016,0.190500
1,4.000,0.101600,0.1143,0.190500
2,4.500,0.114300,0.127,0.215900
3,5.000,0.127000,0.1397,0.215900
4,5.500,0.139700,0.168275,0.228600
5,6.625,0.168275,0.1778,0.269875
6,7.000,0.177800,0.219075,0.269875
7,8.625,0.219075,0.244475,0.311150
8,9.625,0.244475,0.27305,0.349250
9,10.750,0.273050,0.339725,0.381000


In [21]:
def is_water_corrosive(temperature_k:float, 
                       pH:float, 
                       calcium_ion_concentration:float,
                       carbonate_ion_concentration:float,
                       total_dissolved_solids:float) -> float:
    """
    Calculates the Langelier Saturation Index (LSI) of geothermal water.
    
    --------------------------------------------------------
    Input Parameters
        temperature_k: (float) temperature of the water as Kelvin, must be in the range of 273 <= T <=363
        pH: (float) pH of the water
        calcium_ion_concentration: (float) calcium ion concentration as ppm
        carbonate_ion_concentration: (float) carbonate ion concentration as
        total_dissolved_solids: (float) TDS (ppm)

    --------------------------------------------------------
    Returns:
        LSI: (float) Langelier saturation index(LSI)
    """
    #TODO: Check if the temperature is in the valid range
    
    K_potenz = lambda coeff, t: np.dot(coeff, [1, t, -1/t,-np.log10(t), 1/t**2])
    pK2_coeff = [107.8871, .03252849, 5151.79, 38.92561, 563713.9]
    pKsc_coeff = [171.9065,0.077993,2839.319,71.595,0]
    
    pK2 = K_potenz(pK2_coeff, temperature_k)
    logger.info(f'pk2: {pK2}')
    pKsc = K_potenz(pKsc_coeff, temperature_k)
    logger.info(f'pKsc: {pKsc}')
    pCa2 = -np.log10(calcium_ion_concentration/(1000*40.08))
    pHCO3 = -np.log10(carbonate_ion_concentration/(1000*61.0168))
    ionic_strength = total_dissolved_solids/40000
    dielectric_stength = 60954/(temperature_k+116) - 68.937
    alkalinity = 1.82*10**6*(dielectric_stength*temperature_k)**(-1.5)
    pfm = alkalinity*(np.sqrt(ionic_strength)/(1+np.sqrt(ionic_strength))-.31) #activity coefficient for monovalent species at the specified temperature
    pHs = pK2 - pKsc + pCa2 + pHCO3 + 5*pfm #pH of saturation, or the pH at which water is saturated with CaCO3
    LSI = pH - pHs

    return LSI



In [59]:
def calculate_minimum_screen_length(req_flow_rate:float,
                                    hyd_conductivity:float,                                    
                                    bore_lifetime:float,                                    
                                    thickness:float,
                                    is_injection_bore:bool,
                                    drawdown:float=25,
                                    bore_radius:float=.0762,
                                    specific_storage:float=2*10**(-4),) -> float:
    """
    Determine minimum screen length, SL (m)
    based on Eq 4 at http://quebec.hwr.arizona.edu/classes/hwr431/2006/Lab6.pdf
    If Injection bore, SL is multiplied by 2.0, and it is capped at total aquifer thickness
    
    --------------------------------------------------------
    Input Parameters
        i.	Required flow rate, Q (m3/day)
        ii.	Aquifer hydraulic conductivity, K (m/day)
        iii. Bore/project lifetime, t (days)
        iv. Aquifer thickness, Z (m)
        v.	Production or injection bore? 
        vi.	Allowable drawdown, Sw (m) default: Sw = 25 m
        vii.	Bore radius, r (m) , default: r = 0.0762 m (3”)
        viii.	Aquifer specific storage, Ss (m-1) , default: Ss = 2x10-4 m-1

    --------------------------------------------------------
    Returns
        SL: (float) nominal value of the minimum screen length (metres)
        error: (tuple) lower and upper limits representing the uncertainty bounds
    """
    #
    SL = (2.3*req_flow_rate/ (4*np.pi*hyd_conductivity*drawdown)) \
             * np.log10(2.25*hyd_conductivity*bore_lifetime/(bore_radius**2 * specific_storage))
    if is_injection_bore: SL *= 2
    SL = min(SL, thickness)
    error_lower = SL * .9
    error_upper = min(SL * 1.1, thickness)

    return SL, (error_lower, error_upper)

def calculate_casing_friction(depth_to_top_screen:float,
                              req_flow_rate:float,
                              prod_casing_diameter:float,
                              pipe_roughness_coeff:float=100.):
    """
    Estimates production casing friction loss above aquifer

    ----------------------------------------------------------------
    Input Parameters:
        depth_to_top_screen,
        req_flow_rate: as m^3/second,
        prod_casing_diameter,
        pipe_roughness_coeff

    ----------------------------------------------------------------
    Returns:
        hfpc: (float)
    """
    hfpc = (10.67*depth_to_top_screen*req_flow_rate**1.852)/(pipe_roughness_coeff**1.852 * prod_casing_diameter**4.8704)
    return hfpc

#TODO: in the pipeline, should be rounded
def calculate_minimum_screen_diameter(up_hole_friction:float,
                                      screen_length:float,
                                      req_flow_rate:float,
                                      pipe_roughness_coeff:float=100.):
    """
    Determines minimum screen diameter, SDmin (m) using Hazen-Williams equation to ensure up-hole friction < 20 m
    https://en.wikipedia.org/wiki/Hazen%E2%80%93Williams_equation#SI_units
    
    ----------------------------------------------------------------
    Input Parameters:
        up_hole_friction:float must be smaller than 20 else throws exception,
        screen_length:float,
        prod_casing_diameter:float,
        req_flow_rate:float in seconds,
        pipe_roughness_coeff=100
    ----------------------------------------------------------------
    Returns:
        d: float minimum screen diameter
    """
    #TODO: ask where the constant 2 comes from
    try:
        if up_hole_friction > 20:
            raise ValueError("Up-hole friction is too high")
        d = (10.67 * screen_length * req_flow_rate**1.852)\
        / (2*pipe_roughness_coeff**1.852*(20-up_hole_friction))
        d **= 1/4.8704
        return d
    except ValueError as e:
        logger.error(e)
        return None

In [52]:
req_flow_rate_day = 4320
req_flow_rate_sec = req_flow_rate_day/(24*60*60)
hyd_conductivity=5                                
bore_lifetime= 30 * 365                           
thickness = 221 #metre, LTA-LMTA
is_injection_bore= False
drawdown=25,
bore_radius=.0762,
specific_storage=2*10**(-4)



production_SL = calculate_minimum_screen_length(req_flow_rate_day,
                                hyd_conductivity,
                                bore_lifetime,
                                thickness,
                                False
)

injection_SL = calculate_minimum_screen_length(req_flow_rate_day,
                                hyd_conductivity,
                                bore_lifetime,
                                thickness,
                                True
)



logger.info(injection_SL)
logger.info(production_SL)

24-10-2023 INFO (139.4841903499024, (125.53577131491217, 153.43260938489266))
24-10-2023 INFO (69.7420951749512, (62.767885657456084, 76.71630469244633))


In [53]:
def find_nearest_value(val, array, larger_than_val=True):
    """
    Finds the nearest value 
    """
    #casting applied
    valid_vals = array[array >= val] if larger_than_val else array
    nearest_idx = np.argmin(np.abs(valid_vals - val))
    return valid_vals[nearest_idx]
    #TODO: input data as an iterative collection and lookup value
    #returns the nearest value in the collection to the input value
    #numpy abs and argmax, get the nearst index and return the value 
    #this should be a decorator so no need to invoke every step



    #TODO: read casing diameter csv, import the column in meters,
    #use round_algorithm function predefined 
    

def calculate_total_casing():
    #TODO: input: pcd, sd, production screen length, intermediate casing
    #intermediate casing is (LMTA - BSE) * pcd * pi
    #production screen length * pi * sd
    #m2
    pass

def calculate_minimum_interval():
    #TODO: prod/injection : sand face velocity differs
    # sand face velocity 
    # iii.	NGR = net-to-gross ratio for aquifer = 1 (K for Gippsland aquifer units is already averaged)
    # input porosity / flow rate(sec) / injection or prod screen length
    #flow / (0.01 * pi * sl * porosity)
    pass

def round_interval():
    #TODO: next largest standard bit size
    # OHD <SD then OHD = next largest standard bit size
    pass

#injection open hole diameter, produciton open hole diameter
#production screen diameter, injection screen diameter

In [60]:
depth_to_top_screen = 1000
#prod_casing_diameter = 0.1016 #TODO: test with the whole list later
prod_casing_diameter = 0.27305

hpfc = calculate_casing_friction(depth_to_top_screen,
                          req_flow_rate_sec,
                          prod_casing_diameter                                                   
                          )

msd = calculate_minimum_screen_diameter(hpfc,
                                  production_SL[0],
                                  req_flow_rate_sec)

prod_screen_diameter = find_nearest_value(msd, cd['metres'].array,True)

print(prod_screen_diameter)

0.1143


In [ ]:
#STAGE 2: DEFINE PUMP PARAMS
def assign_pump_diameter():
    #TODO: refer to stage 2
    pass
def pump_inlet_depth():
    #inlet or chamber
    #water depth WD, drawdown, margin, dS/dt, lifetime years
    #next step in 2-b
    pass

def pump_diameter_in_meter():
    #TODO: inches to centimeters
    #0.0254 or if you don't like hard-coded conversion factor import module

    pass

def minumum_pump_housing_diameter():
    #TODO: req flow rate(sec), pump diameter
    # formula in the doc
    pass

In [ ]:
#STAGE 3. Determine parameters for each casing state

#QAb : m to base of QA/UTQA aquifer

def calculate_pre_collar_depth():
    #TODO: 3-3
    pass

def calculate_superficial_casing_depth():
    #TODO: 3-4
    pass

def intermediate_casing_depth():
    #TODO: 3-5
    pass

def screen_riser_depths():
    #TODO: 3-6
    pass

#psl , psd

def intermediate_casing_diameter():
    #TODO: 3-9
    #max of two different values
    pass

def is_separate_pump_channel_required():
    #TODO: 3-10
    #injection-well: no
    #prod-well: compare
    #if required: correct 2.3 and 2.2
    pass

def top_intermediate_casing():
    #TODO: pump-chamber presence?
    pass

def superficial_casing_diameter():
    #TODO: 3.4 a???? 
    #if yes, pump chamber? -> scd

    pass

# pre-collar cd and drill-bit next

In [ ]:
#STAGE 4. DEFINE COST COMPONENTS


In [ ]:
#TODO: pipeline for cost calculation and analysis

In [16]:
is_water_corrosive(48.8889+273,
                   8.0,
                   120,
                   100,
                   210)

24-10-2023 INFO pk2: 10.179403392979253
24-10-2023 INFO pKsc: 8.651565532607634


1.8115956832437172

KeyError: "None of [Index([ 0.10680711169442729, 0.005207111694427294, 0.007492888305572709,\n       0.020192888305572712,   0.0328928883055727,  0.06146788830557272,\n        0.07099288830557272,   0.1122678883055727,   0.1376678883055727,\n         0.1662428883055727,   0.2329178883055727,   0.3662678883055727,\n         0.4011928883055727,   0.5027928883055728,   0.6551928883055728],\n      dtype='float64')] are in the [index]"